In [1]:
import pandas as pd
from read_gff import read_gff
data_dir = '../data'
ref_dir = f'{data_dir}/reference'
pgt_201_b1_gff_path = f'{ref_dir}/Pgt_201_B1_GeneCatalog_20191015.gff3'
gff = read_gff(pgt_201_b1_gff_path)
hit_to_gene = {
    'jgi.p|Pgt_201_B1|9243': 'SdhA',
    'jgi.p|Pgt_201_B1|44': 'SdhB',
    'jgi.p|Pgt_201_B1|1581': 'SdhC',
    'jgi.p|Pgt_201_B1|9290': 'SdhD',
    'jgi.p|Pgt_201_B1|17472': 'Cyp51',
}
hits_gff = gff[gff.Name.isin(hit_to_gene) & (gff['type'] == 'gene')]
hits_descendants = set(s for l in hits_gff.Descendants.values for s in l)
hits_gff = gff[gff.Name.isin(hit_to_gene) | gff.ID.isin(hits_descendants)]
gff = pd.read_csv(pgt_201_b1_gff_path, sep='\t', header=None, comment='#')
hits_gff = gff.loc[hits_gff.index]
hits_gff.to_csv(f'{ref_dir}/fungicide_relative_to_pgt_201_b1.gff3', sep='\t', index=None, header=None)

hits_gff.columns = ['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes']
import re
from Bio.Seq import reverse_complement
from Bio.SeqIO import parse
ref = {r.id: str(r.seq) for r in parse(f'{ref_dir}/Pgt_201_B1_AssemblyScaffolds2.fasta', 'fasta')}

# write the fungicide gene sequences as fasta
with open(f'{ref_dir}/fungicide_genes.fa', 'w') as f:
    for _, row in hits_gff.query('type == "gene"').iterrows():
        hit_name =  next(re.finditer('Name=([^;]+);', row.attributes)).group(1)
        gene = hit_to_gene[hit_name]
        if row.strand == '-':
            gene_seq = reverse_complement(ref[row.seqid][row.start - 1:row.end])
        else:
            gene_seq = ref[row.seqid][row.start - 1:row.end]
        f.write('>' + gene + '\n' + gene_seq + '\n')

hit_names = []
for _, row in hits_gff.iterrows():
    if row['type'] == 'gene':
        hit_name = row['attributes'].split('Name=')[1].split(';')[0]
    hit_names.append(hit_name)
genes = [hit_to_gene[hit_name] for hit_name in hit_names]
hits_gff['seqid'] = genes

genes_gff = []
for gene, gene_gff in hits_gff.groupby('seqid'):
    gene_gff = gene_gff[gene_gff['type'].isin({'gene', 'mRNA', 'exon', 'CDS'})].copy()
    # Make relative to gene not chromosome
    old_start = gene_gff.start.values[0]
    old_end = gene_gff.end.values[0]
    length = old_end - old_start + 1
    old_strand = gene_gff.strand.values[0]
    gene_gff['start'] = gene_gff.start + 1 - old_start
    gene_gff['end'] = gene_gff.end + 1 - old_start
    gene_gff_non_mrna_descendants = gene_gff[~ gene_gff['type'].isin({'exon', 'CDS'})].copy()
    gene_gff_mrna_descendants = gene_gff[gene_gff['type'].isin({'exon', 'CDS'})].copy()
    if old_strand == '-':
        # display(gene_gff_mrna_descendants)
        new_start = length - gene_gff_mrna_descendants.end + 1
        new_end = length - gene_gff_mrna_descendants.start + 1
        gene_gff_mrna_descendants['start'] = new_start
        gene_gff_mrna_descendants['end'] = new_end
        # display(gene_gff_mrna_descendants)
    gene_gff = pd.concat([gene_gff_non_mrna_descendants, gene_gff_mrna_descendants])
    gene_gff['strand'] = '+'

    phases = []
    n = 0
    for _, row in gene_gff.iterrows():
        if row.type != 'CDS':
            phases.append('.')
        else:
            phases.append(2 - (n - 1) % 3)
            n += 1 + (row.end - row.start)
    gene_gff['phase'] = phases
    genes_gff.append(gene_gff)

genes_gff = pd.concat(genes_gff)
genes_gff.to_csv('../data/reference/fungicide_genes.gff3', sep='\t', header=None, index=None)